In [1]:
import numpy as np
from cvxpy import *
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [2]:
A = np.array([[  -1.,     0,     0,     0,     0,  0,    1.,     0,     0,    1.,     0,     0,     0,     0],
              [1.003,    -1,     0,     0,     0,  0,     0,    1.,     0, -1.01,    1.,     0,     0,     0],
              [    0, 1.003,    -1,     0,     0,  0,     0,     0,    1.,     0, -1.01,    1.,     0,     0],
              [    0,     0, 1.003,    -1,     0,  0, -1.02,     0,     0,     0,     0, -1.01,    1.,     0],
              [    0,     0,     0, 1.003,    -1,  0,     0, -1.02,     0,     0,     0,     0, -1.01,    1.],
              [    0,     0,     0,     0, 1.003, -1,     0,     0, -1.02,     0,     0,     0,     0, -1.01]])

## Naive Strategy

In [3]:
b_hat = np.array([150, 100, -200, 200, -50, -300])
x_cash = Variable(6)
x_cp = Variable(3)
x_cre = Variable(5)
prob = Problem(Maximize(x_cash[5]), 
               [A[:, :6]*x_cash + A[:, 6:9]*x_cp + \
                A[:, 9:]*x_cre >= \
                b_hat,
                x_cre <= 100, x_cre >=0, x_cp >= 0, x_cash >= 0])
prob.solve()

92.49694914597907

In [4]:
print(x_cash.value.round(4))

[[   0.    ]
 [   0.    ]
 [ 351.9442]
 [   0.    ]
 [   0.    ]
 [  92.4969]]


In [5]:
print(x_cp.value.round(4))

[[ 150.    ]
 [  74.2977]
 [ 177.9035]]


In [6]:
print(x_cre.value.round(4))

[[  0.    ]
 [ 25.7023]
 [  0.    ]
 [  0.    ]
 [ 25.7836]]


## Robust Strategy

In [7]:
# robust problem in the livebook
b_hat = np.array([150, 100, -200, 200, -50, -300])
B=np.diag(b_hat).dot(np.array([[.06,   0,   0,   0,   0,   0],
                            [.02, .06,   0,   0,   0,   0],
                            [  0, .02, .06,   0,   0,   0],
                            [  0,   0, .02, .06,   0,   0],
                            [  0,   0,   0, .02, .06,   0],
                            [  0,   0,   0,   0, .02, .06]]))
x_cash = Variable(6, 1)
x_cp = Variable(3, 1)
x_cre = Variable(5, 1)
prob = Problem(Maximize(x_cash[5]), 
               [A*vstack(x_cash,x_cp,x_cre) >= \
                b_hat + abs(B)*np.ones(6).T,
                x_cre <= 100, x_cre >=0, x_cp >= 0, x_cash >= 0])
prob.solve()

14.288569470597606

In [8]:
print(x_cash.value.round(4))

[[   0.    ]
 [   0.    ]
 [ 377.0489]
 [   0.    ]
 [   0.    ]
 [  14.2886]]


In [9]:
print(x_cp.value.round(4))

[[ 159.    ]
 [  71.6477]
 [ 229.7647]]


In [10]:
print(x_cre.value.round(4))

[[  0.    ]
 [ 36.3523]
 [  0.    ]
 [  0.    ]
 [ 27.0807]]


## Affine Strategy

In [11]:
# linear problem in the livebook
b_hat = np.array([150, 100, -200, 200, -50, -300])
B=np.diag(b_hat).dot(np.array([[.06,   0,   0,   0,   0,   0],
                               [.02, .06,   0,   0,   0,   0],
                               [  0, .02, .06,   0,   0,   0],
                               [  0,   0, .02, .06,   0,   0],
                               [  0,   0,   0, .02, .06,   0],
                               [  0,   0,   0,   0, .02, .06]]))
x_cash = Variable(6, 1)
x_cp = Variable(3, 1)
x_cre = Variable(5, 1)
X_Cash = Variable(6, 6)
X_Cp = Variable(3, 6)
X_Cre = Variable(5, 6)

constraints = [A*vstack(x_cash,x_cp,x_cre) >= b_hat + \
               sum_entries(abs(B - A*vstack(X_Cash,X_Cp,X_Cre)), 
                           axis=1),
               vstack(x_cash,x_cp,x_cre) >= \
               sum_entries(abs(vstack(X_Cash,X_Cp,X_Cre)), axis=1),
                x_cre + sum_entries(abs(X_Cre), axis=1) <= 100]
for i in range(6):
    constraints += [X_Cash[i, i:] == 0]
for i in range(3):
    constraints += [X_Cp[i, i:] == 0]
for i in range(5):
    constraints += [X_Cre[i, i:] == 0]

prob = Problem(Maximize(x_cash[5]-sum(abs(X_Cash[-1, :]))), 
               constraints)
prob.solve()

14.288569491253206

In [12]:
print(x_cash.value.round(4))

[[   0.    ]
 [   2.539 ]
 [ 380.0183]
 [  11.0139]
 [   9.2145]
 [  19.7831]]


In [13]:
print(x_cp.value.round(4))

[[ 159.    ]
 [  74.2119]
 [ 220.5392]]


In [14]:
print(x_cre.value.round(4))

[[  0.    ]
 [ 34.6441]
 [  4.9162]
 [ 11.2591]
 [ 40.6723]]


In [15]:
print(X_Cash.value.round(4))

[[ 0.      0.     -0.     -0.     -0.     -0.    ]
 [-2.539  -0.     -0.      0.     -0.     -0.    ]
 [-0.5575  2.412   0.      0.     -0.      0.    ]
 [-2.4966  3.0407 -5.4767 -0.     -0.     -0.    ]
 [-1.9932  1.8879 -2.9699  2.3635  0.     -0.    ]
 [-0.3188  0.9143 -1.0454  0.316   2.9001  0.    ]]


In [16]:
print(X_Cp.value.round(4))

[[-0.      0.     -0.     -0.      0.      0.    ]
 [ 1.7868  0.     -0.      0.      0.      0.    ]
 [ 1.4496 -3.3814 -0.      0.      0.      0.    ]]


In [17]:
print(X_Cre.value.round(4))

[[-0.     -0.      0.      0.      0.      0.    ]
 [-2.6428  0.      0.     -0.      0.      0.    ]
 [-2.4282  2.488  -0.     -0.      0.     -0.    ]
 [-4.7544  3.9545 -2.5501  0.      0.      0.    ]
 [-2.8159  3.6097 -1.0086  1.7194 -0.      0.    ]]
